Notebook that uses a pythonized version of projposolar.pro (originally by Pat Irwin) to make maps of planet location, latitude, longitude, emission angle, etc, and then query those maps to extract spectra along a line of latitude w/ bins in cosine(emission angle) space. Currently written specifically for NAIC hyperspectral data cubes of Jupiter, but could be modified for other datasets.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from scipy.interpolate import interp1d
import math
from uncertainties import ufloat
from uncertainties.umath import *
from uncertainties import unumpy
from scipy import interpolate

Define projposolar programs (a bit messy and butchered at this point, but this combo of programs works for the moment)

In [ ]:
def interceptellip(a,b,alpha,beta,gamma,x0,y0,z0):
    '''
    ; **********************************************************
    ; procedure to find the intercepts (if any) between the line
    ;
    ;  (x-x0)       (y-y0)      (z-z0)
    ;  ------  =    ------   =  ------
    ;  alpha         beta       gamma
    ; 
    ; and the ellipsoid
    ;
    ;
    ;  x^2    y^2    z^2   
    ;  --- +  --- +  ---  = 1
    ;  a^2    a^2    b^2
    ;
    ; Input variables
    ;       a       real    ellipsoid semi-major axis
    ;       b       real    ellipsoid semi-minor axis
    ;       alpha   real    line x-gradient
    ;       beta    real    line y-gradient
    ;       gamma   real    line z-gradient
    ;       x0      real    line x-intercept
    ;       y0      real    line y-intercept
    ;       z0      real    line z-intercept
    ;
    ; Output variables
    ;       iflag   integer Set to 1 if line intercepts, set to -1  otherwise
    ;       x(2)    real    x-intercepts
    ;       y(2)    real    y-intercepts
    ;       z(2)    real    z-intercepts
    ;
    ; Pat Irwin     11/2/07
    ; Python conversion - Emma Dahl 3-8-19
    ; **********************************************************
    '''
    a1 = 1.0/a**2 + (beta/(a*alpha))**2 + (gamma/(b*alpha))**2

    b1 = (-2*x0*beta**2/alpha**2 + 2*beta*y0/alpha)/a**2
    b1 = b1 + (-2*x0*gamma**2/alpha**2 + 2*gamma*z0/alpha)/b**2
    
    c1 = ((beta*x0/alpha)**2 - 2*beta*y0*x0/alpha + y0**2)/a**2
    c1 = c1 + ((gamma*x0/alpha)**2 - 2*gamma*x0*z0/alpha + z0**2)/b**2 -1
        
    #;print,a1,1.0/a**2 + (gamma/(b*alpha))**2
    #;print,b1,2*gamma*z0/(alpha*b**2)
    #;print,c1,(y0/a)**2 + (z0/b)**2 -1
    
    xtest = b1**2 - 4*a1*c1

    x = np.zeros(2)
    y = np.zeros(2)
    z = np.zeros(2)
        
    if xtest > 0.0:
        iflag = 1
        x[0] = (-b1 + np.sqrt(xtest))/(2*a1)
        x[1] = (-b1 - np.sqrt(xtest))/(2*a1)
        y[0] = y0 + (beta/alpha)*(x[0]-x0)
        y[1] = y0 + (beta/alpha)*(x[1]-x0)
        z[0] = z0 + (gamma/alpha)*(x[0]-x0)
        z[1] = z0 + (gamma/alpha)*(x[1]-x0)

        # testing to see if solution is on ellipsoid
        test=np.ndarray(len(x))
        for i in range(0,len(x)):
            test[i] = (x[i]/a)**2 + (y[i]/a)**2 + (z[i]/b)**2
        xtest1 = abs(test[0]-1.0)
        xtest2 = abs(test[1]-1.0)
        
        err = 1e-5
        
        if xtest1 > err or xtest2 > err:
            print 'Problem in interceptellip - solution not on ellipsoid'
            print 'Test =',test

    return iflag,x,y,z

In [ ]:
def projposolar(Re,obl,epsilon,latsol,lonsol,se_lon,eoff,poff):
    '''
    ; ************************************************************
    ; Procedure to find latitude and longitude and zenith angle of
    ; intercept between line and ellipsoid
    ;
    ; Input variables
    ;       Re      real    Equatorial Radius (arcsec)
    ;       obl     real    Planetary oblateness
    ;       epsilon real    Sub-observer (planetocentric) latitude
    ;       latsol  real    Sub-solar planetocentric latitude
    ;       lonsol  real    longitude difference between sub-solar and sub-observer
    ;                       points.
    ;       se_lon  real    Sub-observer longitude # added by Emma
    ;       eoff    real    equatorial offset of beam (arcsec)
    ;       poff    real    polar offset of beam (arcsec)
    ;
    ; Output variables
    ;       iflag   integer Set to 1 if real intercept, -1 otherwise
    ;       xlat    real    Planetocentric latitude
    ;       longitude real  xlon+se_lon, offset of longitude added to sub-observer longitude # added by Emma
    ;       xlon    real    Longitude
    ;       zen     real    Zenith angle
    ;       szen    real    Solar zenith angle
    ;       aphi    real    local azimuth angle between sun and observer
    ;
    ; Pat Irwin     11/2/07
    ; Python conversion - Emma Dahl 3/8/19
    ; ************************************************************
    '''
    
    dtr = np.pi/180.0 # radians/degrees
    Rp = Re*(1.0-obl)
    
    x0 = 0.0
    y0 = eoff
    z0 = poff/np.cos(epsilon*dtr)
    
    alpha = np.sin(np.pi/2.0 - epsilon*dtr)
    beta = 0.0
    gamma = np.cos(np.pi/2.0 - epsilon*dtr)
        
    iflag,x,y,z = interceptellip(Re,Rp,alpha,beta,gamma,x0,y0,z0)
    
    xlat = 0.0
    xlon = 0.0
    zen = 0.0
    
    if iflag > 0:
        # if real intercept, find lat, long, and zenith
        
        # find distance along line of sight
        lambdaa = (x-x0)/alpha
        if lambdaa[0] > lambdaa[1]:
            inear = 0
        else:
            inear = 1
                
        x1 = x[inear]
        y1 = y[inear]
        z1 = z[inear]
        
        r = np.sqrt(x1**2 + y1**2 + z1**2)
        
        theta = np.arccos(z1/r)
        xlat = 90.0 - theta/dtr
                
        #; convert to planetographic latitude
        #; xlat = np.arctan(((Re/Rp)**2)*np.tan(xlat*dtr))/dtr
        
        cphi = x1/(r*np.sin(theta))
        
        if cphi > 1.0:
            cphi = 1.0
        if cphi < -1.0:
            cphi = -1.0
                    
        phi = np.arccos(cphi)
        if y1 < 0.0:
            phi = -phi
        xlon = phi/dtr        
        
        # Finding aphi, zen, szen - don't mess with these, want to still output them
        
        v1 = np.zeros(3)
        v2 = np.zeros(3)
        v3 = np.zeros(3)
        
        # v1 is normal vector of point observed
        v1[0] = x1/r
        v1[1] = y1/r
        v1[2] = z1/r
        
        v2[0] = alpha
        v2[1] = beta
        v2[2] = gamma
        
        summ = 0.0
        
        for i in range(0,3):
            summ += v1[i]*v2[i]
        zen = np.arccos(summ)/dtr
        
        # Finding aphi
        
        alphasol = np.sin(np.pi/2.0 - latsol*dtr)*np.cos(lonsol*dtr)
        betasol =  np.sin(np.pi/2.0 - latsol*dtr)*np.sin(lonsol*dtr)
        gammasol = np.cos(np.pi/2 - latsol*dtr)
        v3[0]=alphasol
        v3[1]=betasol
        v3[2]=gammasol

        summ = 0.0
        for i in range(0,3):
            summ += v1[i]*v3[i]
        szen = np.arccos(summ)/dtr
        
        cphase = 0.0
        
        for i in range(0,3):
            cphase += v2[i]*v3[i]
            
        a = np.cos(zen*dtr)*np.cos(szen*dtr)
        b = np.sin(zen*dtr)*np.sin(szen*dtr)
        
        if b == 0.0:
            aphi = 180.0
        else:
            cphi = (cphase-a)/b
            aphi = 180.0-np.arccos(cphi)/dtr
            
    longitude = se_lon-xlon # offset of longitude added to sub-observer longitude
    # Remember that system III longitude increases to the west
    
    #return xlat,longitude
    return iflag,xlat,longitude,xlon,zen,szen,aphi

In [ ]:
def interceptellip(a,b,alpha,beta,gamma,x0,y0,z0):
    '''
    ; **********************************************************
    ; procedure to find the intercepts (if any) between the line
    ;
    ;  (x-x0)       (y-y0)      (z-z0)
    ;  ------  =    ------   =  ------
    ;  alpha         beta       gamma
    ; 
    ; and the ellipsoid
    ;
    ;
    ;  x^2    y^2    z^2   
    ;  --- +  --- +  ---  = 1
    ;  a^2    a^2    b^2
    ;
    ; Input variables
    ;       a       real    ellipsoid semi-major axis
    ;       b       real    ellipsoid semi-minor axis
    ;       alpha   real    line x-gradient
    ;       beta    real    line y-gradient
    ;       gamma   real    line z-gradient
    ;       x0      real    line x-intercept
    ;       y0      real    line y-intercept
    ;       z0      real    line z-intercept
    ;
    ; Output variables
    ;       iflag   integer Set to 1 if line intercepts, set to -1  otherwise
    ;       x(2)    real    x-intercepts
    ;       y(2)    real    y-intercepts
    ;       z(2)    real    z-intercepts
    ;
    ; Pat Irwin     11/2/07
    ; Python conversion - Emma Dahl 3-8-19
    ; **********************************************************
    '''
    a1 = 1.0/a**2 + (beta/(a*alpha))**2 + (gamma/(b*alpha))**2

    b1 = (-2*x0*beta**2/alpha**2 + 2*beta*y0/alpha)/a**2
    b1 = b1 + (-2*x0*gamma**2/alpha**2 + 2*gamma*z0/alpha)/b**2
    
    c1 = ((beta*x0/alpha)**2 - 2*beta*y0*x0/alpha + y0**2)/a**2
    c1 = c1 + ((gamma*x0/alpha)**2 - 2*gamma*x0*z0/alpha + z0**2)/b**2 -1
        
    #;print,a1,1.0/a**2 + (gamma/(b*alpha))**2
    #;print,b1,2*gamma*z0/(alpha*b**2)
    #;print,c1,(y0/a)**2 + (z0/b)**2 -1
    
    xtest = b1**2 - 4*a1*c1

    x = np.zeros(2)
    y = np.zeros(2)
    z = np.zeros(2)
        
    if xtest > 0.0:
        iflag = 1
        error_value=0
        x[0] = (-b1 + np.sqrt(xtest))/(2*a1)
        x[1] = (-b1 - np.sqrt(xtest))/(2*a1)
        y[0] = y0 + (beta/alpha)*(x[0]-x0)
        y[1] = y0 + (beta/alpha)*(x[1]-x0)
        z[0] = z0 + (gamma/alpha)*(x[0]-x0)
        z[1] = z0 + (gamma/alpha)*(x[1]-x0)

        # testing to see if solution is on ellipsoid
        test=np.ndarray(len(x))
        for i in range(0,len(x)):
            test[i] = (x[i]/a)**2 + (y[i]/a)**2 + (z[i]/b)**2
        xtest1 = abs(test[0]-1.0)
        xtest2 = abs(test[1]-1.0)
        
        err = 1e-5
        
        if xtest1 > err or xtest2 > err:
            print 'Problem in interceptellip - solution not on ellipsoid'
            print 'Test =',test
            
    else:
        iflag = -1

        

    return iflag,x,y,z

def iflag_only(Re,obl,epsilon,latsol,lonsol,se_lon,eoff,poff):
    '''
    ; ************************************************************
    ; Procedure to find latitude and longitude and zenith angle of
    ; intercept between line and ellipsoid
    '''
    
    dtr = np.pi/180.0 # radians/degrees
    Rp = Re*(1.0-obl)
    
    x0 = 0.0
    y0 = eoff
    z0 = poff/np.cos(epsilon*dtr)
    
    alpha = np.sin(np.pi/2.0 - epsilon*dtr)
    beta = 0.0
    gamma = np.cos(np.pi/2.0 - epsilon*dtr)
        
    iflag,x,y,z = interceptellip(Re,Rp,alpha,beta,gamma,x0,y0,z0)
    
    #return xlat,longitude
    return iflag

In [ ]:
def projposolar_iflag_is_1(Re,obl,epsilon,latsol,lonsol,se_lon,eoff,poff):
    '''
    ; ************************************************************
    ; Procedure to find latitude and longitude and zenith angle of
    ; intercept between line and ellipsoid
    ;
    ; Input variables
    ;       Re      real    Equatorial Radius (arcsec)
    ;       obl     real    Planetary oblateness
    ;       epsilon real    Sub-observer (planetocentric) latitude
    ;       latsol  real    Sub-solar planetocentric latitude
    ;       lonsol  real    longitude difference between sub-solar and sub-observer
    ;                       points.
    ;       se_lon  real    Sub-observer longitude # added by Emma
    ;       eoff    real    equatorial offset of beam (arcsec)
    ;       poff    real    polar offset of beam (arcsec)
    ;
    ; Output variables
    ;       iflag   integer Set to 1 if real intercept, -1 otherwise
    ;       xlat    real    Planetocentric latitude
    ;       longitude real  xlon+se_lon, offset of longitude added to sub-observer longitude # added by Emma
    ;       xlon    real    Longitude
    ;       zen     real    Zenith angle
    ;       szen    real    Solar zenith angle
    ;       aphi    real    local azimuth angle between sun and observer
    ;
    ; Pat Irwin     11/2/07
    ; Python conversion - Emma Dahl 3/8/19
    ; ************************************************************
    '''
    
    dtr = np.pi/180.0 # radians/degrees
    Rp = Re*(1.0-obl)
    
    x0 = 0.0
    y0 = eoff
    z0 = poff/np.cos(epsilon*dtr)
    
    alpha = np.sin(np.pi/2.0 - epsilon*dtr)
    beta = 0.0
    gamma = np.cos(np.pi/2.0 - epsilon*dtr)
        
    iflag,x,y,z = interceptellip(Re,Rp,alpha,beta,gamma,x0,y0,z0)
    iflag = 1
    
    xlat = 0.0
    xlon = 0.0
    zen = 0.0
    
    if iflag > 0:
        # if real intercept, find lat, long, and zenith
        
        # find distance along line of sight
        lambdaa = (x-x0)/alpha
        if lambdaa[0] > lambdaa[1]:
            inear = 0
        else:
            inear = 1
                
        x1 = x[inear]
        y1 = y[inear]
        z1 = z[inear]
        
        r = np.sqrt(x1**2 + y1**2 + z1**2)
        
        theta = np.arccos(z1/r)
        xlat = 90.0 - theta/dtr
                
        #; convert to planetographic latitude
        #; xlat = np.arctan(((Re/Rp)**2)*np.tan(xlat*dtr))/dtr
        
        cphi = x1/(r*np.sin(theta))
        
        if cphi > 1.0:
            cphi = 1.0
        if cphi < -1.0:
            cphi = -1.0
                    
        phi = np.arccos(cphi)
        if y1 < 0.0:
            phi = -phi
        xlon = phi/dtr        
        
        # Finding aphi, zen, szen - don't mess with these, want to still output them
        
        v1 = np.zeros(3)
        v2 = np.zeros(3)
        v3 = np.zeros(3)
        
        # v1 is normal vector of point observed
        v1[0] = x1/r
        v1[1] = y1/r
        v1[2] = z1/r
        
        v2[0] = alpha
        v2[1] = beta
        v2[2] = gamma
        
        summ = 0.0
        
        for i in range(0,3):
            summ += v1[i]*v2[i]
        zen = np.arccos(summ)/dtr
        
        # Finding aphi
        
        alphasol = np.sin(np.pi/2.0 - latsol*dtr)*np.cos(lonsol*dtr)
        betasol =  np.sin(np.pi/2.0 - latsol*dtr)*np.sin(lonsol*dtr)
        gammasol = np.cos(np.pi/2 - latsol*dtr)
        v3[0]=alphasol
        v3[1]=betasol
        v3[2]=gammasol

        summ = 0.0
        for i in range(0,3):
            summ += v1[i]*v3[i]
        szen = np.arccos(summ)/dtr
        
        cphase = 0.0
        
        for i in range(0,3):
            cphase += v2[i]*v3[i]
            
        a = np.cos(zen*dtr)*np.cos(szen*dtr)
        b = np.sin(zen*dtr)*np.sin(szen*dtr)
        
        if b == 0.0:
            aphi = 180.0
        else:
            cphi = (cphase-a)/b
            aphi = 180.0-np.arccos(cphi)/dtr
            
    longitude = se_lon-xlon # offset of longitude added to sub-observer longitude
    # Remember that system III longitude increases to the west
    
    #return xlat,longitude
    return iflag,xlat,longitude,xlon,zen,szen,aphi

Extract basic geometric information (e.g. sub-observer lat and long) from NAIC Jupiter cube for each frame of cube (since Jupiter is rotating). Will be used as input for projposolar.

In [ ]:
cube = fits.open('/Volumes/external_hd_2/PAL/emmas_stuff/PJ19_data/PJ19_cube4_v2.fits')
# data directory, list
directory = '/Volumes/external_hd_2/PAL/emmas_stuff/PJ19_data/jup4_navigated_v2/'
imlist = np.loadtxt('/Volumes/external_hd_2/PAL/emmas_stuff/PJ19_data/juplist4',dtype=str)

# DEFINE PLATE SCALE HERE
platescale = 0.08531 #"/pix - post-2018 camera
#platescale = 0.10593 #"/pix - pre-2018 camera

# values from horizons - not dependent on observation
eq_rad_km = 71492 # jupiter's equatorial radius in km
oblateness = 0.06487 # from wikipedia

# make functions for finding planetocentric latitude from planetographic latitude
# file from http://w.astro.berkeley.edu/~mikewong/doc/kmdeg_0.25.txt
coordinates = np.loadtxt('/Users/dahlek/Desktop/jup_geo_coordinates',dtype=float)

graph = [] # planetographic coordinates
centric = [] # planetocentric coordinates

for i in range(0,len(coordinates)):
    graph.append(coordinates[i][0])
    centric.append(coordinates[i][1])
    
# centric as a function of graph
centric_func = interp1d(graph,centric)

k = 0

# array of info. wl (nm), equatorial radius (arcsec), oblateness, epsilon, latsol, lonsol, se_lon
geo_info = np.zeros((cube[0].header['naxis3'],7))

# Find inputs from Jupiter data that has already been run through disk_corr. (need planet center!)
for i in range(0,len(imlist)):
    im = fits.open(directory+imlist[i])
    if im[0].header['rfon'] == 1:
        print imlist[i],im[0].header['lambda'],'nm'
        geo_info[k,0] = im[0].header['lambda']
        eq_rad_arcsec = round(np.arctan(eq_rad_km/im[0].header['PE_RNG'])*206265,4)
        print 'Re',eq_rad_arcsec
        geo_info[k,1] = eq_rad_arcsec
        print 'obl:',oblateness
        geo_info[k,2] = oblateness
        if im[0].header['SE_LAT'] < 0:
            print 'epsilon:', -centric_func(abs(im[0].header['SE_LAT'])) # degrees
            geo_info[k,3] = -centric_func(abs(im[0].header['SE_LAT']))
        else:
            print 'epsilon:', centric_func(im[0].header['SE_LAT']) # degrees
            geo_info[k,3] = centric_func(abs(im[0].header['SE_LAT']))
        if im[0].header['SS_LAT'] < 0:
            print 'latsol:', -centric_func(abs(im[0].header['SS_LAT'])) # degrees
            geo_info[k,4] = -centric_func(abs(im[0].header['SS_LAT']))
        else:
            print 'latsol:', centric_func(im[0].header['SS_LAT']) # degrees
            geo_info[k,4] = centric_func(abs(im[0].header['SS_LAT']))
        print 'lonsol:',im[0].header['SS_LON']-im[0].header['SE_LON']
        geo_info[k,5] = im[0].header['SS_LON']-im[0].header['SE_LON']
        print 'se_lon:',im[0].header['SE_LON']
        geo_info[k,6] = im[0].header['SE_LON']
        k+=1 # a check to make sure list of info and cube are the same length        
        
        print ' '
    im.close()
    
if k != cube[0].header['naxis3']:
    print 'ERROR! Unequal numbers of images in list for geometric info and in image cube.'
    
cube.close()

Make iflag map - will be 1 where planet is on the frame, 0 otherwise.

In [ ]:
# define regions to ignore to slightly speed up the mapping process
pixel_border_to_ignore = 100 # rows on top and bottom of image to ignore while searching for chosen_lat
column_to_ignore = 100 # columns to ignore on left and right of image

j=0 # calculate iflag map for the 0th image in the cube. shouldn't have to change this if Jupiter is aligned correctly in the hyperspectral cube; it'll be at the same spot for each frame.

row_position = [] # to be filled w/ row position of chosen_lat
column_position = [] # to be filled w/ column position corresponding to chosen_lat value

iflag_map = np.zeros((cube[0].header['naxis1'],cube[0].header['naxis2']))

#plt.matshow(cube[0].data[j],origin='lower')
#plt.show()

# loop through each pixel column, find the row pixel closest to chosen_latitude:
for i in range(column_to_ignore,cube[0].header['NAXIS2']-column_to_ignore):
    
    # loop through pixels in column (ignoring borders defined by pixel_border_to_ignore), calculate 
    for k in range(pixel_border_to_ignore, cube[0].header['NAXIS2']-pixel_border_to_ignore):
        
        # use x,y pixel locations to calculate offset from center, which will be used as input for projposolar
        #print i,k # column, row
        long_offset_pixel = i - cube[0].header['NAXIS2']/2
        lat_offset_pixel = k - cube[0].header['NAXIS2']/2
        long_offset_arcsec = long_offset_pixel*platescale
        lat_offset_arcsec = lat_offset_pixel*platescale
        #print lat_offset_arcsec, long_offset_arcsec, 'lat/long offsets in arcseonds'
        
        # test iflag:
        #iflag = iflag_only(eq_rad_arcsec, oblateness, epsilon, se_lon, latsol, lonsol, long_offset_arcsec, lat_offset_arcsec)
        iflag = iflag_only(geo_info[j,1],geo_info[j,2],geo_info[j,3],geo_info[j,4],geo_info[j,5],geo_info[j,6], long_offset_arcsec, lat_offset_arcsec)
        #print i,k,iflag
        iflag_map[k][i] = iflag
        
# make whole map 0 except for planet
iflag_map[np.where(iflag_map != 1)] = 0

plt.matshow(iflag_map)
plt.colorbar()
plt.show()

In [ ]:
# save iflag map
np.savetxt('/Volumes/external_hd_2/PAL/emmas_stuff/PJ19_data/spectra_extraction_maps/iflag_cube4',iflag_map)

Determine minimum mu cutoff

In [ ]:
# make a mu map to determine cutoff minimum mu value

wl_offset = np.loadtxt('/Users/dahlek/Desktop/post_jan_2018_tuning_curve')
wavel_spxs = wl_offset[:,2]
# load cube:
cube = fits.open('/Volumes/external_hd_2/PAL/emmas_stuff/PJ19_data/PJ19_cube4_v2.fits')

f = 1 # image cube index

print wavel_spxs[f]
emission_angle_array = np.zeros((cube[0].header['NAXIS1'],cube[0].header['NAXIS2']))
mu_array = np.zeros((cube[0].header['NAXIS1'],cube[0].header['NAXIS2']))


for i in range(0,len(iflag_map)):
    for j in range(0,len(iflag_map[i])):
        if iflag_map[i][j] == 1:
            # calculate the emission angle of that pixel and save it to emission_angle_array
            long_offset_pixel = j - cube[0].header['NAXIS2']/2
            lat_offset_pixel = i - cube[0].header['NAXIS2']/2
            long_offset_arcsec = long_offset_pixel*platescale
            lat_offset_arcsec = lat_offset_pixel*platescale

            iflag,latitude_final,longitude_final,xlon,zen,szen,aphi = projposolar_iflag_is_1(geo_info[f,1],geo_info[f,2],geo_info[f,3],geo_info[f,4],geo_info[f,5],geo_info[f,6], long_offset_arcsec, lat_offset_arcsec)

            emission_angle_array[i][j] = zen
            mu_array[i][j] = np.cos(zen*0.0174533) # in radians
            
cube.close()

In [ ]:
plt.matshow(mu_array)
plt.title('Mu')
plt.colorbar()
plt.show()

In [ ]:
# determine bin size based on seeing (derive from seeing_finder.ipynb) 
seeing = 0.9085 # arcseconds

# lower mu limit = 2x seeing value
mu_min_arcsec = seeing*2
mu_min_pix = int(round(mu_min_arcsec/platescale)) # minimum mu cutoff in pixels
print mu_min_pix, 'pixels from edge of planet'

In [ ]:
for i in range(0,len(mu_array)):
    # find the first row with a value in it
    if mu_array[i].any() != 0:
        break
        
# find the column
j = np.where(mu_array[i] != 0)[0][len(np.where(mu_array[i] != 0)[0])/2]

plt.matshow(mu_array)
plt.plot(j,i,'ro')
plt.plot(j,i+mu_min_pix,'bo')
plt.show()

mu_min_value = mu_array[i+mu_min_pix][j]
print 'minimum mu value = ',mu_min_value

Decide latitude range to sample

In [ ]:
# make maps of latitude and counts

wl_offset = np.loadtxt('/Users/dahlek/Desktop/post_jan_2018_tuning_curve')
wavel_spxs = wl_offset[:,2]
# load cube:
cube = fits.open('/Volumes/external_hd_2/PAL/emmas_stuff/PJ19_data/PJ19_cube4_v2.fits')


f = 1 # image cube index

print wavel_spxs[f]
spx_array = np.zeros((cube[0].header['NAXIS1'],cube[0].header['NAXIS2']))
lat_array = np.zeros((cube[0].header['NAXIS1'],cube[0].header['NAXIS2']))


for i in range(0,len(iflag_map)):
    for j in range(0,len(iflag_map[i])):
        if iflag_map[i][j] == 1:
            # calculate the emission angle of that pixel and save it to emission_angle_array
            long_offset_pixel = j - cube[0].header['NAXIS2']/2
            lat_offset_pixel = i - cube[0].header['NAXIS2']/2
            long_offset_arcsec = long_offset_pixel*platescale
            lat_offset_arcsec = lat_offset_pixel*platescale

            iflag,latitude_final,longitude_final,xlon,zen,szen,aphi = projposolar_iflag_is_1(geo_info[f,1],geo_info[f,2],geo_info[f,3],geo_info[f,4],geo_info[f,5],geo_info[f,6], long_offset_arcsec, lat_offset_arcsec)

            spx_array[i][j] = cube[0].data[f][i][j]
            lat_array[i][j] = latitude_final

cube.close()

In [ ]:
# decide latitude range; plot them out on an image

lat_min = -1.5
lat_max = 0.5

f = 0 # cube frame index

cut_array = np.zeros((cube[0].header['NAXIS1'],cube[0].header['NAXIS2']))
overlay_array = np.copy(spx_array)

for i in range(0,len(iflag_map)):
    for j in range(0,len(iflag_map[i])):
        if iflag_map[i][j] == 1:
            if lat_array[i][j] > lat_min and lat_array[i][j] < lat_max:
                cut_array[i][j] = spx_array[i][j]
                overlay_array[i][j] = 0
                
                
# plot the slice by itself, and the whole planet minus the slice for context

plt.matshow(cut_array,origin='low')
plt.show()

plt.matshow(overlay_array,origin='low')
plt.title(str(round(wavel_spxs[15],2))+' nm')
plt.show()

Pull and save lists (or, 1D arrays) of counts, emission angle, mu, latitude, longitude, solar emission angle, azimuth angle. (might be nice to make these arrays in the future - if so, will have to change how indexing is done in spectral extraction cell. For now, good enough.)

In [ ]:
# load NAIC wavelengths
wl_offset = np.loadtxt('/Users/dahlek/Desktop/post_jan_2018_tuning_curve')
wavel_spxs = wl_offset[:,2]

# load cube:
cube = fits.open('/Users/dahlek/Desktop/PJ19/PJ19_cube4_v2.fits')

# will look for pixels between these lat and longs
lat_min = -1.5
lat_max = 0.5

for f in range(0,len(wavel_spxs)):
    
    print 'Finding data lists for',wavel_spxs[f],' nm...'
    spxs = [] # counts
    mu_spxs = [] # cosine of zenith emission anlgle, in radians
    lat_spxs = [] # latitude (planetocentric)
    long_spxs = [] # longitude
    solar_spxs = [] # solar emission angle
    azimuth_spxs = [] # azimuth angle
    lcm_spxs = [] # central meridian (should be the same value for each frame)

    for i in range(0,len(iflag_map)):
        for j in range(0,len(iflag_map[i])):
            if iflag_map[i][j] == 1:
                # calculate the emission angle of that pixel and save it to emission_angle_array
                long_offset_pixel = j - cube[0].header['NAXIS2']/2
                lat_offset_pixel = i - cube[0].header['NAXIS2']/2
                long_offset_arcsec = long_offset_pixel*platescale
                lat_offset_arcsec = lat_offset_pixel*platescale

                iflag,latitude_final,longitude_final,xlon,zen,szen,aphi = projposolar_iflag_is_1(geo_info[f,1],geo_info[f,2],geo_info[f,3],geo_info[f,4],geo_info[f,5],geo_info[f,6], long_offset_arcsec, lat_offset_arcsec)

                spxs.append(cube[0].data[f][i][j])
                mu_spxs.append(np.cos(zen*0.0174533))
                lat_spxs.append(latitude_final)
                long_spxs.append(longitude_final)
                solar_spxs.append(szen)
                azimuth_spxs.append(aphi)
                lcm_spxs.append(geo_info[f,6]) # assuming sub-observer longitude is lcm

    # save lists
    np.savetxt('/Volumes/external_hd_2/PAL/emmas_stuff/PJ19_data/spectra_extraction_maps/'+str(f)+'_spx', spxs)
    np.savetxt('/Volumes/external_hd_2/PAL/emmas_stuff/PJ19_data/spectra_extraction_maps/'+str(f)+'_mu_spx', mu_spxs)
    np.savetxt('/Volumes/external_hd_2/PAL/emmas_stuff/PJ19_data/spectra_extraction_maps/'+str(f)+'_lat_spx', lat_spxs)
    np.savetxt('/Volumes/external_hd_2/PAL/emmas_stuff/PJ19_data/spectra_extraction_maps/'+str(f)+'_long_spx', long_spxs)
    np.savetxt('/Volumes/external_hd_2/PAL/emmas_stuff/PJ19_data/spectra_extraction_maps/'+str(f)+'_solar_spx', solar_spxs)
    np.savetxt('/Volumes/external_hd_2/PAL/emmas_stuff/PJ19_data/spectra_extraction_maps/'+str(f)+'_azi_spx', azimuth_spxs)
    np.savetxt('/Volumes/external_hd_2/PAL/emmas_stuff/PJ19_data/spectra_extraction_maps/'+str(f)+'_lcm_spx', lcm_spxs)
    
    
cube.close()

Extract spectra from a given mu interval, within the previously defined latitude and mu limits

Currently pulls spectra from either side of the meridian, plus a central spectrum. Likely a bug is still in here related to the total number of bins being even or odd.

In [ ]:
wl_offset = np.loadtxt('/Users/dahlek/Desktop/post_jan_2018_tuning_curve')
wavel_spxs = wl_offset[:,2]
# load cube:
cube = fits.open('/Users/dahlek/Desktop/PJ19/PJ19_cube4_v2.fits')

# where lists were saved previously
geo_list_directory = '/Volumes/external_hd_2/PAL/emmas_stuff/PJ19_data/spectra_extraction_maps/'

# number of NAIC wavelengths
n_wavel = 241

# define bins w/ the size of bin in mu-space
#d_mu = 0.05 # original value
#d_mu = 0.115
d_mu = 0.14
mu_min = np.copy(mu_min_value)
mu_max = 1.0
w_mu = 0.1
n_mu = int((mu_max-mu_min)/d_mu)
n_bins = n_mu*2-1

# define bins based on n_mu (number of bins)
'''
mu_min = np.copy(mu_min_value)
mu_max = 1.0
n_mu = 11
d_mu = ((mu_max-mu_min)/n_mu)
w_mu = 2*d_mu
'''

# will look for pixels between these lat and longs
lat_min = -1.5
lat_max = 0.5

# make empty arrays for binning section
extracted_spectrum_sum = np.zeros((n_bins,n_wavel))
extracted_sum = np.zeros((n_bins,n_wavel))
extracted_lat_sum = np.zeros((n_bins,n_wavel))
extracted_long_sum = np.zeros((n_bins,n_wavel))
extracted_emiss_sum = np.zeros((n_bins,n_wavel))
extracted_solar_sum = np.zeros((n_bins,n_wavel))
extracted_azi_sum = np.zeros((n_bins,n_wavel))

for f in range(0,len(wavel_spxs)):

    # load lists of geometric info for each wavelength/frame
    spxs = np.loadtxt(geo_list_directory+str(f)+'_spx')
    mu_spxs = np.loadtxt(geo_list_directory+str(f)+'_mu_spx')
    lat_spxs = np.loadtxt(geo_list_directory+str(f)+'_lat_spx')
    long_spxs = np.loadtxt(geo_list_directory+str(f)+'_long_spx')
    solar_spxs = np.loadtxt(geo_list_directory+str(f)+'_solar_spx')
    azimuth_spxs = np.loadtxt(geo_list_directory+str(f)+'_azi_spx')
    lcm_spxs = np.loadtxt(geo_list_directory+str(f)+'_lcm_spx')
    
    # once lists are loaded, extract spectra
    i_wavel = np.copy(f)
    wavel = wavel_spxs[f]
    
    print 'Extracting spectra for',wavel,'nm...'
    
    h = 0 # bin index, for assigning values in arrays
    
    print 'west'
    for i_mu in range(0,n_mu-1):
        # print mu index
        #print 'i_mu:',i_mu
        mu_0 = mu_min + float(d_mu*i_mu)
        mu_1 = mu_min + float(d_mu*i_mu) + w_mu
        # print mu limits
        #print 'mu_0, mu_1:',mu_0,mu_1
        
        n_spx = len(spxs) # number of pixels in location arrays/lists
        for i_spx in range(0,n_spx):
            # pull relavent values from lists
            radiance = spxs[i_spx]
            mu_spx = mu_spxs[i_spx]
            emiss_spx = (180/math.pi)*np.arccos(mu_spx)
            lat_spx = lat_spxs[i_spx]
            long_spx = long_spxs[i_spx]
            solar_spx = solar_spxs[i_spx]
            azimuth_spx = azimuth_spxs[i_spx]
            lcm_spx = lcm_spxs[i_spx]
            
            #only interested in spectra over EZ
            #if lat_spx < lat_max or lat_spx > lat_min:
            if lat_spx < lat_max and lat_spx > lat_min:
                #if wavel_spx == wavel and mu_spx > mu_0 and mu_spx < mu_1 and long_spx < lcm_spx:
                #long_spx < lcm_spx - east
                #long_spx > lcm_spx - west
                if mu_spx > mu_0 and mu_spx < mu_1 and long_spx > lcm_spx:
                    extracted_spectrum_sum[h,i_wavel] = extracted_spectrum_sum[h,i_wavel] + radiance
                    extracted_lat_sum[h,i_wavel] = extracted_lat_sum[h,i_wavel] + lat_spx
                    extracted_long_sum[h,i_wavel] = extracted_long_sum[h,i_wavel] + long_spx
                    extracted_emiss_sum[h,i_wavel] = extracted_emiss_sum[h,i_wavel] + emiss_spx
                    extracted_solar_sum[h,i_wavel] = extracted_solar_sum[h,i_wavel] + solar_spx
                    extracted_azi_sum[h,i_wavel] = extracted_azi_sum[h,i_wavel] + azimuth_spx
                    
                    extracted_sum[h,i_wavel] = extracted_sum[h,i_wavel] + 1.
            #print extracted_lat_sum[i_mu,i_wavel]
            
        h+=1
        
    print 'center'
    i_mu = n_mu
    mu_0 = mu_min + float(d_mu*i_mu)
    mu_1 = mu_min + float(d_mu*i_mu) + w_mu
    # print mu limits
    #print 'mu_0, mu_1:',mu_0,mu_1

    n_spx = len(spxs) # number of pixels in location arrays/lists
    for i_spx in range(0,n_spx):
        # pull relavent values from lists
        radiance = spxs[i_spx]
        mu_spx = mu_spxs[i_spx]
        emiss_spx = (180/math.pi)*np.arccos(mu_spx)
        lat_spx = lat_spxs[i_spx]
        long_spx = long_spxs[i_spx]
        solar_spx = solar_spxs[i_spx]
        azimuth_spx = azimuth_spxs[i_spx]
        lcm_spx = lcm_spxs[i_spx]

        #only interested in spectra over EZ
        #if lat_spx < lat_max or lat_spx > lat_min:
        if lat_spx < lat_max and lat_spx > lat_min:
            #if wavel_spx == wavel and mu_spx > mu_0 and mu_spx < mu_1 and long_spx < lcm_spx:
            #long_spx < lcm_spx - east
            #long_spx > lcm_spx - west
            if mu_spx > mu_0:
                extracted_spectrum_sum[h,i_wavel] = extracted_spectrum_sum[h,i_wavel] + radiance
                extracted_lat_sum[h,i_wavel] = extracted_lat_sum[h,i_wavel] + lat_spx
                extracted_long_sum[h,i_wavel] = extracted_long_sum[h,i_wavel] + long_spx
                extracted_emiss_sum[h,i_wavel] = extracted_emiss_sum[h,i_wavel] + emiss_spx
                extracted_solar_sum[h,i_wavel] = extracted_solar_sum[h,i_wavel] + solar_spx
                extracted_azi_sum[h,i_wavel] = extracted_azi_sum[h,i_wavel] + azimuth_spx

                extracted_sum[h,i_wavel] = extracted_sum[h,i_wavel] + 1.
        #print extracted_lat_sum[i_mu,i_wavel]
    
    h+=1
    
    
    print 'east'
    for i_mu in range(-n_mu,-1):
        #print -1*i_mu
        mu_0 = mu_min + float(d_mu*i_mu*-1)
        mu_1 = mu_min + float(d_mu*i_mu*-1) - w_mu
        #print 'mu_0, mu_1:',mu_0,mu_1
        
        n_spx = len(spxs) # number of pixels in location arrays/lists
        for i_spx in range(0,n_spx):
            # pull relavent values from lists
            radiance = spxs[i_spx]
            mu_spx = mu_spxs[i_spx]
            emiss_spx = (180/math.pi)*np.arccos(mu_spx)
            lat_spx = lat_spxs[i_spx]
            long_spx = long_spxs[i_spx]
            solar_spx = solar_spxs[i_spx]
            azimuth_spx = azimuth_spxs[i_spx]
            lcm_spx = lcm_spxs[i_spx]

            #only interested in spectra over EZ
            #if lat_spx < lat_max or lat_spx > lat_min:
            if lat_spx < lat_max and lat_spx > lat_min:
                #if wavel_spx == wavel and mu_spx > mu_0 and mu_spx < mu_1 and long_spx < lcm_spx:
                #long_spx < lcm_spx - east
                #long_spx > lcm_spx - west
                if mu_spx < mu_0 and mu_spx > mu_1 and long_spx < lcm_spx:
                    extracted_spectrum_sum[h,i_wavel] = extracted_spectrum_sum[h,i_wavel] + radiance
                    extracted_lat_sum[h,i_wavel] = extracted_lat_sum[h,i_wavel] + lat_spx
                    extracted_long_sum[h,i_wavel] = extracted_long_sum[h,i_wavel] + long_spx
                    extracted_emiss_sum[h,i_wavel] = extracted_emiss_sum[h,i_wavel] + emiss_spx
                    extracted_solar_sum[h,i_wavel] = extracted_solar_sum[h,i_wavel] + solar_spx
                    extracted_azi_sum[h,i_wavel] = extracted_azi_sum[h,i_wavel] + azimuth_spx

                    extracted_sum[h,i_wavel] = extracted_sum[h,i_wavel] + 1.
            #print extracted_lat_sum[i_mu,i_wavel]

        h+=1    
        
            
    # Take the average by dividing the sums of each by the total number of points
    extracted_average_spectra = extracted_spectrum_sum/extracted_sum
    extracted_average_lat = extracted_lat_sum/extracted_sum
    extracted_average_long = extracted_long_sum/extracted_sum
    extracted_average_emiss = extracted_emiss_sum/extracted_sum
    extracted_average_solar = extracted_solar_sum/extracted_sum
    extracted_average_azi = extracted_azi_sum/extracted_sum

    
cube.close()

In [1]:
# save everything

np.savetxt('/Users/dahlek/Desktop/PJ19/extracted_spectra_cube4/average_spectra_n1.5p0.5_v2align_d_mu_0.14',extracted_average_spectra)

np.savetxt('/Users/dahlek/Desktop/PJ19/extracted_spectra_cube4/average_lat_n1.5p0.5_d_mu_0.14',extracted_average_lat)

np.savetxt('/Users/dahlek/Desktop/PJ19/extracted_spectra_cube4/average_long_n1.5p0.5_d_mu_0.14',extracted_average_long)

np.savetxt('/Users/dahlek/Desktop/PJ19/extracted_spectra_cube4/average_emiss_n1.5p0.5_d_mu_0.14',extracted_average_emiss)

np.savetxt('/Users/dahlek/Desktop/PJ19/extracted_spectra_cube4/average_solar_n1.5p0.5_d_mu_0.14',extracted_average_solar)

np.savetxt('/Users/dahlek/Desktop/PJ19/extracted_spectra_cube4/average_azi_n1.5p0.5_d_mu_0.14',extracted_average_azi)

NameError: name 'np' is not defined